# Include libraries

In [1]:
import os
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sksurv.column import encode_categorical

## data preprocessing

In [2]:
from sklearn import preprocessing
from sklearn_pandas import DataFrameMapper

def data_processing(data_df):
    data_df_x = data_df.drop(['LOC', 'UID', 'Mortality', 'SurvivalDays'], axis=1)

    data_df_y = data_df[['Mortality', 'SurvivalDays']]

    X_temp = data_df_x[(data_df.LOC == '3') | (data_df.LOC == '2') | (data_df.LOC == '6')]
    y_temp = data_df_y[(data_df.LOC == '3') | (data_df.LOC == '2') | (data_df.LOC == '6')]
    X_df_train, X_df_val, y_df_train, y_df_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=369)

    X_df_test_kao = data_df_x[data_df.LOC == '8']
    y_df_test_kao = data_df_y[data_df.LOC == '8']

    categorical_columns = ['Sex', 'AF', 'DM', 'HTN', 'Dyslipidemia', 'CHF', 'Smoking',
                           'Cancer.before.adm']
    
    numerical_columns = np.setdiff1d(data_df_x.columns, categorical_columns).tolist()

    categorical_ix = [data_df_x.columns.get_loc(col) for col in categorical_columns]
    numerical_ix =  np.setdiff1d(list(range(0, len(data_df_x.columns))), categorical_ix).tolist()

    scaler = preprocessing.StandardScaler()

    standardize = [([col], scaler) for col in numerical_columns]
    leave = [(col, None) for col in categorical_columns]

    x_mapper = DataFrameMapper(standardize + leave)

    X_df_train = pd.DataFrame(data=x_mapper.fit_transform(X_df_train),
                              columns=numerical_columns+categorical_columns,
                              index=X_df_train.index)

    X_df_val = pd.DataFrame(data=x_mapper.fit_transform(X_df_val),
                            columns=numerical_columns+categorical_columns,
                            index=X_df_val.index)

    X_df_test_kao = pd.DataFrame(data=x_mapper.fit_transform(X_df_test_kao),
                                 columns=numerical_columns+categorical_columns,
                                 index=X_df_test_kao.index)

    X_df_train = encode_categorical(X_df_train, columns=categorical_columns)
    X_df_val = encode_categorical(X_df_val, columns=categorical_columns)
    X_df_test_kao = encode_categorical(X_df_test_kao, columns=categorical_columns)
    
    return X_df_train, X_df_val, y_df_train, y_df_val, X_df_test_kao, y_df_test_kao

# DL setting

In [3]:
import torch
import torchtuples as tt
from pycox.evaluation import EvalSurv

get_target = lambda df: (df['SurvivalDays'].values, df['Mortality'].values)

### 3_1 dataset

In [4]:
data_1 = pd.read_csv(os.path.join('..', '..', 'data', '(v3_1)STROKE_VITAL_SIGN_MICE.csv'))
X_train_1, X_val_1, y_train_1, y_val_1, X_test_kao_1, y_test_kao_1 = data_processing(data_1)
# for DLs
X_test_kao_1 = X_test_kao_1.values.astype('float32')
durations_test_kao_1, events_test_kao_1 = get_target(y_test_kao_1)
# for RSF
y_train_cox_1 =  y_train_1.copy()
y_train_cox_1.loc[:, 'Mortality'] = y_train_cox_1['Mortality'].astype(bool)
y_train_cox_1 = np.array(list(y_train_cox_1.to_records(index=False)))
y_test_kao_cox_1 =  y_test_kao_1.copy()
y_test_kao_cox_1.loc[:, 'Mortality'] = y_test_kao_cox_1['Mortality'].astype(bool)
y_test_kao_cox_1 = np.array(list(y_test_kao_cox_1.to_records(index=False)))

In [5]:
### 3_2 dataset

In [6]:
data_2 = pd.read_csv(os.path.join('..', '..', 'data', '(v3_2)STROKE_VITAL_SIGN_MICE.csv'))
X_train_2, X_val_2, y_train_2, y_val_2, X_test_kao_2, y_test_kao_2 = data_processing(data_2)
# for DLs
X_test_kao_2 = X_test_kao_2.values.astype('float32')
durations_test_kao_2, events_test_kao_2 = get_target(y_test_kao_2)
# for RSF
y_train_cox_2 =  y_train_2.copy()
y_train_cox_2.loc[:, 'Mortality'] = y_train_cox_2['Mortality'].astype(bool)
y_train_cox_2 = np.array(list(y_train_cox_2.to_records(index=False)))
y_test_kao_cox_2 =  y_test_kao_2.copy()
y_test_kao_cox_2.loc[:, 'Mortality'] = y_test_kao_cox_2['Mortality'].astype(bool)
y_test_kao_cox_2 = np.array(list(y_test_kao_cox_2.to_records(index=False)))

## Deepsur

In [7]:
from pycox.models import CoxPH

def Deepsur(Xtrain, Xval, Ytrain, Yval):
    # preprocessing data
    Xtrain = Xtrain.values.astype('float32')
    Xval = Xval.values.astype('float32')
    Ytrain = get_target(Ytrain)
    Yval = get_target(Yval)
    val = Xval, Yval
    
    # parameters
    in_features = Xtrain.shape[1]
    num_nodes = [46, 32, 8]
    out_features = 1
    batch_norm = True
    dropout = 0.1
    output_bias = False
    batch_size = 256
    if Xtrain.shape[0]%batch_size == 1:
        batch_size = batch_size - 1
    epochs = 100
    callbacks = [tt.callbacks.EarlyStopping(patience=20)]
    verbose = False
    
    # network
    net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm,
                                  dropout, output_bias=output_bias)
    model = CoxPH(net, tt.optim.Adam)
    
    # train
    lrfinder = model.lr_finder(Xtrain, Ytrain, batch_size, tolerance=10)
    best_lr = lrfinder.get_best_lr()
    #_ = lrfinder.plot()
#     best_lr = 0.089
    model.optimizer.set_lr(best_lr)
    log = model.fit(Xtrain, Ytrain, batch_size, epochs, callbacks, verbose,
                    val_data=val, val_batch_size=batch_size)
    return model
    

In [8]:
seed = 369
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

deep_sur_1 = Deepsur(X_train_1, X_val_1, y_train_1, y_val_1)

In [9]:
# prediction
_ = deep_sur_1.compute_baseline_hazards()
surv_kao_1 = deep_sur_1.predict_surv_df(X_test_kao_1)
# evaluation
ev_kao_1 = EvalSurv(surv_kao_1, durations_test_kao_1, events_test_kao_1, censor_surv='km')
print('3_1 Kao C-index = %.3f' %(ev_kao_1.concordance_td()))

3_1 Kao C-index = 0.819


In [10]:
seed = 369
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

deep_sur_2 = Deepsur(X_train_2, X_val_2, y_train_2, y_val_2)

In [11]:
# prediction
_ = deep_sur_2.compute_baseline_hazards()
surv_kao_2 = deep_sur_2.predict_surv_df(X_test_kao_2)
# evaluation
ev_kao_2 = EvalSurv(surv_kao_2, durations_test_kao_2, events_test_kao_2, censor_surv='km')
print('3_2 Kao C-index = %.3f' %(ev_kao_2.concordance_td()))

3_2 Kao C-index = 0.815


## DeepHit

In [12]:
from pycox.models import DeepHitSingle

def Deephit(Xtrain, Xval, Ytrain, Yval):
    # preprocessing data
    Xtrain = Xtrain.values.astype('float32')
    Xval = Xval.values.astype('float32')
    
    # label transforms
    num_durations = 480
    labtrans = DeepHitSingle.label_transform(num_durations)
    Ytrain = labtrans.fit_transform(*get_target(Ytrain))
    Yval = labtrans.transform(*get_target(Yval))
    train = (Xtrain, Ytrain)
    val = (Xval, Yval)
    
    # parameters
    in_features = Xtrain.shape[1]
    num_nodes = [46, 32, 8]
    out_features = labtrans.out_features
    batch_norm = True
    dropout = 0.1
    output_bias = False
    batch_size = 256
    if Xtrain.shape[0]%batch_size == 1:
        batch_size = batch_size - 1
    epochs = 100
    callbacks = [tt.callbacks.EarlyStopping(patience=20)]
    verbose = False
    
    # network
    net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm,
                                  dropout, output_bias=output_bias)
    
    model = DeepHitSingle(net, tt.optim.Adam, alpha=0.2, sigma=0.1, duration_index=labtrans.cuts)

    lrfinder = model.lr_finder(Xtrain, Ytrain, batch_size, tolerance=10)
    best_lr = lrfinder.get_best_lr()
    #print(best_lr)
    #_ = lrfinder.plot()
    #best_lr = 0.089
    model.optimizer.set_lr(best_lr)
    
    log = model.fit(Xtrain, Ytrain, batch_size, epochs, callbacks, verbose,
                    val_data=val, val_batch_size=batch_size)
    
    return model

In [13]:
# Seed
seed = 369
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

deep_hit = Deephit(X_train_1, X_val_1, y_train_1, y_val_1)

In [14]:
# prediction
hit_surv_1 = deep_hit.predict_surv_df(X_test_kao_1)
# evaluation
ev_kao_1 = EvalSurv(hit_surv_1, durations_test_kao_1, events_test_kao_1, censor_surv='km')
print('3_1 Kao C-index = %.3f' %(ev_kao_1.concordance_td()))

3_1 Kao C-index = 0.808


In [15]:
# Seed
seed = 369
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

deep_hit = Deephit(X_train_2, X_val_2, y_train_2, y_val_2)

In [16]:
# prediction
hit_surv_2 = deep_hit.predict_surv_df(X_test_kao_2)
# evaluation
ev_kao_2 = EvalSurv(hit_surv_2, durations_test_kao_2, events_test_kao_2, censor_surv='km')
print('3_2 Kao C-index = %.3f' %(ev_kao_2.concordance_td()))

3_2 Kao C-index = 0.802


## RandomSurvivalForest

In [17]:
from sksurv.ensemble import RandomSurvivalForest

rsf_1 = RandomSurvivalForest(n_estimators=100,
                           min_samples_split=10,
                           min_samples_leaf=15,
                           max_features="sqrt",
                           n_jobs=-1,
                           random_state=369)

rsf_1.fit(X_train_1, y_train_cox_1)
print('3_1 Kao C-index = %.3f' %(rsf_1.score(X_test_kao_1, y_test_kao_cox_1)))

3_1 Kao C-index = 0.817


In [18]:
# Predicted risk scores
import eli5
from eli5.sklearn import PermutationImportance
feature_names = X_train_1.columns.values
perm_1 = PermutationImportance(rsf_1, n_iter=5, random_state=369)
perm_1.fit(X_test_kao_1, y_test_kao_cox_1)
eli5.show_weights(perm_1, feature_names=feature_names)

Weight,Feature
0.0349 ± 0.0036,Age
0.0326 ± 0.0040,eNIHSS
0.0213 ± 0.0022,Creatinine
0.0133 ± 0.0018,Cancer.before.adm=1.0
0.0132 ± 0.0032,V
0.0085 ± 0.0007,Mean.HR
0.0063 ± 0.0017,BMI
0.0043 ± 0.0008,Duration_day
0.0027 ± 0.0008,M
0.0025 ± 0.0008,CHOL


In [19]:
eli5.format_as_dataframe(eli5.explain_weights(perm_1, feature_names=feature_names, top=100)).to_csv('rfs_FS(3_1).csv', index=False)

In [20]:
from sksurv.ensemble import RandomSurvivalForest

rsf_2 = RandomSurvivalForest(n_estimators=100,
                           min_samples_split=10,
                           min_samples_leaf=15,
                           max_features="sqrt",
                           n_jobs=-1,
                           random_state=369)

rsf_2.fit(X_train_2, y_train_cox_2)
print('3_2 Kao C-index = %.3f' %(rsf_2.score(X_test_kao_2, y_test_kao_cox_2)))

3_2 Kao C-index = 0.814


In [21]:
# Predicted risk scores
import eli5
from eli5.sklearn import PermutationImportance
feature_names = X_train_2.columns.values
perm_2 = PermutationImportance(rsf_2, n_iter=5, random_state=369)
perm_2.fit(X_test_kao_2, y_test_kao_cox_2)
eli5.show_weights(perm_2, feature_names=feature_names)

Weight,Feature
0.0354 ± 0.0043,eNIHSS
0.0349 ± 0.0026,Age
0.0151 ± 0.0019,Cancer.before.adm=1.0
0.0113 ± 0.0033,V
0.0065 ± 0.0013,eGFR
0.0049 ± 0.0015,MeanHR.G
0.0043 ± 0.0004,Duration_day
0.0038 ± 0.0014,M
0.0028 ± 0.0008,BMI
0.0020 ± 0.0007,CHOL


In [22]:
eli5.format_as_dataframe(eli5.explain_weights(perm_2, feature_names=feature_names, top=100)).to_csv('rfs_FS(3_2).csv', index=False)